In [1]:
%load_ext autoreload
%autoreload 2
%env DAGSTER_HOME=/home/santi/personal/EdMachina/dagster_repo/storage

env: DAGSTER_HOME=/home/santi/personal/EdMachina/dagster_repo/storage


In [2]:
raw_data.head()

NameError: name 'raw_data' is not defined

In [4]:
from dagster_repo import defs
from dagster import AssetKey


In [5]:
with defs.get_asset_value_loader() as loader:
    raw_data = loader.load_asset_value(
        AssetKey("raw_student_data")
    )


No dagster instance configuration file (dagster.yaml) found at /home/santi/personal/EdMachina/dagster_repo/storage. Defaulting to loading and storing all metadata with /home/santi/personal/EdMachina/dagster_repo/storage. If this is the desired behavior, create an empty dagster.yaml file in /home/santi/personal/EdMachina/dagster_repo/storage.


2024-07-29 19:55:28 -0300 - dagster - DEBUG - system - Loading file from: /home/santi/personal/EdMachina/dagster_repo/storage/storage/raw_student_data using PickledObjectFilesystemIOManager...


In [6]:
import pandas as pd

In [7]:
raw_data.columns

Index(['particion', 'periodo', 'nota_final_materia', 'fecha_mesa_epoch',
       'nombre_examen', 'nota_parcial', 'assignment_id', 'ass_name',
       'ass_created_at', 'ass_due_at', 'ass_unlock_at', 'ass_lock_at',
       'points_possible', 'ass_name_sub', 'sub_uuid', 'score',
       'submission_type', 's_submitted_at', 's_graded_at', 's_created_at',
       'user_uuid', 'course_uuid', 'course_name', 'legajo'],
      dtype='object')

In [ ]:
raw_data.info()

In [8]:
base_columns = ['periodo', 'user_uuid','course_uuid']
examns_columns = base_columns + ['fecha_mesa_epoch', 'nombre_examen', 'nota_parcial']
assigments_columns = base_columns + ['assignment_id', 'ass_name', 'ass_created_at', 'ass_due_at','ass_unlock_at', 'ass_lock_at', 'points_possible', 'ass_name_sub']
submissions_columns = base_columns + ['sub_uuid', 'score','submission_type', 's_submitted_at', 's_graded_at', 's_created_at']
final_grade = raw_data[base_columns + ['nota_final_materia']].drop_duplicates()
examns_df = raw_data.copy()[examns_columns]
assigments_df = raw_data.copy()[assigments_columns]
submissions_df = raw_data.copy()[submissions_columns]

In [9]:
examns_df.dropna(inplace=True)
pivot = pd.pivot_table(examns_df, index=['user_uuid','course_uuid','periodo'], columns='nombre_examen', values='nota_parcial', fill_value=0).reset_index(drop=False)
pivot['x'] = pivot.apply(lambda x:  x['RECUPERATORIO PRIMER PARCIAL(20)'] if x['RECUPERATORIO PRIMER PARCIAL(20)'] > 0 else  x['PRIMER PARCIAL(20)'], axis=1)
pivot

nombre_examen,user_uuid,course_uuid,periodo,INTEGRADOR(30),PRIMER PARCIAL(20),RECUPERATORIO PRIMER PARCIAL(20),RECUPERATORIO SEGUNDO PARCIAL(20),SEGUNDO PARCIAL(20),x
0,01103370-3046-4bc0-a8f8-85858ec0ec88,2dd2ca01-a672-4eca-aca7-6fbc8cd01953,1-2022,0.0,7.0,0.0,0.0,7.0,7.0
1,01103370-3046-4bc0-a8f8-85858ec0ec88,a8c25bca-c5e5-4225-b51e-f61468d20386,1-2022,0.0,8.0,0.0,0.0,9.0,8.0
2,01103370-3046-4bc0-a8f8-85858ec0ec88,bf8d7a98-903d-4dea-b4d2-275394aa0a06,1-2022,0.0,8.0,0.0,0.0,7.0,8.0
3,01103370-3046-4bc0-a8f8-85858ec0ec88,d17e238c-c4c6-40d7-8085-801268d6d723,1-2022,0.0,9.0,0.0,0.0,9.0,9.0
4,01103370-3046-4bc0-a8f8-85858ec0ec88,ef072dc2-cf38-4330-a10b-1786cd20da44,1-2022,0.0,7.0,0.0,0.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...
2758,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,12bb6bfa-8b01-46a8-8cc2-e0e84785fa1e,1-2022,0.0,0.0,8.0,0.0,8.0,8.0
2759,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,2b274b15-83bb-4629-941e-fdc573af4e0b,1-2022,0.0,9.0,0.0,8.0,0.0,9.0
2760,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,68005d22-90dc-42f6-b90a-409714481cbc,1-2022,0.0,7.0,0.0,7.0,4.0,7.0
2761,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,c78af3b4-4574-4a82-8c41-45195ce43384,1-2022,7.0,0.0,0.0,0.0,0.0,0.0


In [10]:
dataset = pd.merge(pivot, final_grade, on=['user_uuid','course_uuid','periodo'])
dataset


,user_uuid,course_uuid,periodo,INTEGRADOR(30),PRIMER PARCIAL(20),RECUPERATORIO PRIMER PARCIAL(20),RECUPERATORIO SEGUNDO PARCIAL(20),SEGUNDO PARCIAL(20),x,nota_final_materia
0,01103370-3046-4bc0-a8f8-85858ec0ec88,2dd2ca01-a672-4eca-aca7-6fbc8cd01953,1-2022,0.0,7.0,0.0,0.0,7.0,7.0,7.0
1,01103370-3046-4bc0-a8f8-85858ec0ec88,a8c25bca-c5e5-4225-b51e-f61468d20386,1-2022,0.0,8.0,0.0,0.0,9.0,8.0,9.0
2,01103370-3046-4bc0-a8f8-85858ec0ec88,bf8d7a98-903d-4dea-b4d2-275394aa0a06,1-2022,0.0,8.0,0.0,0.0,7.0,8.0,8.0
3,01103370-3046-4bc0-a8f8-85858ec0ec88,d17e238c-c4c6-40d7-8085-801268d6d723,1-2022,0.0,9.0,0.0,0.0,9.0,9.0,9.0
4,01103370-3046-4bc0-a8f8-85858ec0ec88,ef072dc2-cf38-4330-a10b-1786cd20da44,1-2022,0.0,7.0,0.0,0.0,6.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...
2758,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,12bb6bfa-8b01-46a8-8cc2-e0e84785fa1e,1-2022,0.0,0.0,8.0,0.0,8.0,8.0,8.0
2759,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,2b274b15-83bb-4629-941e-fdc573af4e0b,1-2022,0.0,9.0,0.0,8.0,0.0,9.0,9.0
2760,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,68005d22-90dc-42f6-b90a-409714481cbc,1-2022,0.0,7.0,0.0,7.0,4.0,7.0,7.0
2761,ff6b6af0-b7d7-42f4-821d-afd5ee6ef9a5,c78af3b4-4574-4a82-8c41-45195ce43384,1-2022,7.0,0.0,0.0,0.0,0.0,0.0,7.0
